## Baresql : an attempt to make a better sql handling of Python datas

Tips:
 - zz$$ is to create temporarly a table zz in SQLite from Python list or Dataframe zz
 - :z (otherwise $z legacy) is a variable to use in Sqlite query from Python Variable z
 - use "PERSIST zz" to keep a Python table in the Database between 2 calls
 - use pydef to create python function in SQLite:
    . "pydef" is the beginning of the function, first semicolumn ";" the end of the function
    . every in & out paramater of a Python function must be a string
 
Other packages in same domain: ipython-sql, pandasql, sqlite-bro, db.py, jupyterlab-sql
   

In [1]:
from baresql import baresql
bsql = baresql.baresql()
bsqldf = lambda q: bsql.df(q, dict(globals(),**locals()))

datas = (1 , 2 , 3 , 4, 5 )
toto = 7
titi = 9
sql="""
pydef py_sqrt(s):
    "sqrt function from Python Math standard library"
    import math as py_math
    return ("%s" %  py_math.sqrt(float(s)));
with delta(x) as (select  c0  - (avg(c0) over() )    from datas$$)
 select py_sqrt(sum(x*x) / (count(*) -1)) , $toto as just_to_check , :titi from delta"""
bsqldf(sql)

*** def py_sqrt(s):
    "sqrt function from Python Math standard library"
    import math as py_math
    return ("%s" %  py_math.sqrt(float(s))) ***


,py_sqrt(sum(x*x) / (count(*) -1)),just_to_check,:titi
0,1.5811388300841898,7,9


In [2]:
import statistics
statistics.stdev(datas)

1.5811388300841898

In [3]:
# defining a pydef function
bsqldf("pydef py_quad(s): return ('%s' %  float(s)**4)")
bsqldf("select py_quad(2)")

*** def py_quad(s): return ('%s' %  float(s)**4) ***


,py_quad(2)
0,16.0


In [14]:
# use persist to keep tables between 2 request
datas = (1 , 2 , 3 , 4, 5 )
datas2 =(1, 2, 3, 4, 5, 6, 7)
datas3 =(1, 2, 3,)

bsqldf("PERSIST datas, datas2; select * from datas3$$")

,index,c0
0,0,1.0
1,1,2.0
2,2,3.0


In [15]:
bsqldf("select * from sqlite_master")

,type,name,tbl_name,rootpage,sql
0,table,datas,datas,4,"CREATE TABLE ""datas"" (\n""index"" INTEGER,\n ""c..."
1,index,ix_datas_index,datas,5,"CREATE INDEX ""ix_datas_index""ON ""datas"" (""index"")"
2,table,datas2,datas2,7,"CREATE TABLE ""datas2"" (\n""index"" INTEGER,\n ""..."
3,index,ix_datas2_index,datas2,3,"CREATE INDEX ""ix_datas2_index""ON ""datas2"" (""in..."
